# Jour 2 - Exercice 3 : Modèles avancés de Machine Learning

## Objectifs
- Explorer des modèles de machine learning plus avancés
- Implémenter XGBoost pour la classification
- Réaliser une optimisation des hyperparamètres
- Comparer les performances avec les modèles précédents
- Visualiser les résultats avec Plotly

## Introduction

Dans ce notebook, nous allons approfondir notre analyse en utilisant des modèles de machine learning plus avancés. Nous nous concentrerons particulièrement sur XGBoost, un algorithme puissant basé sur le gradient boosting, et nous explorerons comment optimiser ses hyperparamètres pour améliorer les performances. Nous utiliserons toujours le jeu de données de satisfaction des passagers.

## 1. Chargement et préparation des données

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Bibliothèques pour le machine learning
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Métriques d'évaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc

# XGBoost
import xgboost as xgb

# Pour afficher plus de colonnes dans les DataFrames
pd.set_option('display.max_columns', 30)

In [ ]:
# Chargement du jeu de données
df = pd.read_csv('../../data/passenger_satisfaction/train.csv')

# Affichage des premières lignes
df.head()

In [ ]:
# Vérification de la distribution de la variable cible
target_counts = df['Satisfaction'].value_counts()

fig = px.pie(values=target_counts.values, 
             names=target_counts.index, 
             title='Distribution de la satisfaction des passagers',
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_traces(textinfo='percent+label')
fig.show()

## 2. Prétraitement des données

Nous allons reprendre les étapes de prétraitement des notebooks précédents pour préparer nos données pour les modèles avancés.

In [4]:
# Suppression de la colonne ID qui n'est pas utile pour la prédiction
df = df.drop('ID', axis=1)

# Conversion de la variable cible en valeurs numériques
target_mapping = {'neutral or dissatisfied': 0, 'satisfied': 1}
df['Satisfaction'] = df['Satisfaction'].map(target_mapping)

# Séparation des caractéristiques et de la variable cible
X = df.drop('Satisfaction', axis=1)
y = df['Satisfaction']

# Division en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Identification des caractéristiques numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Caractéristiques numériques: {numeric_features}")
print(f"Caractéristiques catégorielles: {categorical_features}")

In [ ]:
# Création d'un pipeline de prétraitement
# Pour les caractéristiques numériques: imputation des valeurs manquantes et standardisation
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pour les caractéristiques catégorielles: imputation des valeurs manquantes et encodage one-hot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Combinaison des transformateurs dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Application du prétraitement aux données d'entraînement et de validation
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

print(f"Forme des données d'entraînement après prétraitement: {X_train_preprocessed.shape}")
print(f"Forme des données de validation après prétraitement: {X_val_preprocessed.shape}")

## 3. Feature Engineering avancé

Ajoutons quelques features supplémentaires pour améliorer les performances de nos modèles.

In [ ]:
# Création d'une copie des données originales pour le feature engineering
X_train_fe = X_train.copy()
X_val_fe = X_val.copy()

# 1. Création d'une feature pour la satisfaction moyenne des services
service_columns = [
    'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness'
]

X_train_fe['Average_Service_Rating'] = X_train_fe[service_columns].mean(axis=1)
X_val_fe['Average_Service_Rating'] = X_val_fe[service_columns].mean(axis=1)

# 2. Création d'une feature pour le retard total (départ + arrivée)
X_train_fe['Total_Delay'] = X_train_fe['Departure Delay in Minutes'] + X_train_fe['Arrival Delay in Minutes'].fillna(0)
X_val_fe['Total_Delay'] = X_val_fe['Departure Delay in Minutes'] + X_val_fe['Arrival Delay in Minutes'].fillna(0)

# 3. Création d'une feature binaire pour indiquer s'il y a eu un retard
X_train_fe['Has_Delay'] = (X_train_fe['Total_Delay'] > 0).astype(int)
X_val_fe['Has_Delay'] = (X_val_fe['Total_Delay'] > 0).astype(int)

# 4. Création d'une feature pour la variance des évaluations de service
X_train_fe['Service_Rating_Variance'] = X_train_fe[service_columns].var(axis=1)
X_val_fe['Service_Rating_Variance'] = X_val_fe[service_columns].var(axis=1)

# Mise à jour des listes de caractéristiques
new_numeric_features = ['Average_Service_Rating', 'Total_Delay', 'Has_Delay', 'Service_Rating_Variance']
numeric_features_fe = numeric_features + new_numeric_features
categorical_features_fe = categorical_features

print(f"Nouvelles caractéristiques numériques: {new_numeric_features}")
print(f"Nombre total de caractéristiques numériques après feature engineering: {len(numeric_features_fe)}")

In [ ]:
# Création d'un nouveau pipeline de prétraitement avec les nouvelles features
numeric_transformer_fe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_fe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor_fe = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_fe, numeric_features_fe),
        ('cat', categorical_transformer_fe, categorical_features_fe)
    ])

# Application du prétraitement aux données d'entraînement et de validation avec feature engineering
X_train_fe_preprocessed = preprocessor_fe.fit_transform(X_train_fe)
X_val_fe_preprocessed = preprocessor_fe.transform(X_val_fe)

print(f"Forme des données d'entraînement après feature engineering et prétraitement: {X_train_fe_preprocessed.shape}")
print(f"Forme des données de validation après feature engineering et prétraitement: {X_val_fe_preprocessed.shape}")

## 4. Fonction d'évaluation des modèles

Nous allons créer une fonction pour évaluer les performances des différents modèles de manière cohérente.

In [9]:
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble d'entraînement et de validation
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calcul des métriques
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    train_precision = precision_score(y_train, y_train_pred)
    val_precision = precision_score(y_val, y_val_pred)
    
    train_recall = recall_score(y_train, y_train_pred)
    val_recall = recall_score(y_val, y_val_pred)
    
    train_f1 = f1_score(y_train, y_train_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    # Affichage des métriques
    print(f"Performances du modèle {model_name}:")
    print(f"Accuracy - Train: {train_accuracy:.4f}, Validation: {val_accuracy:.4f}")
    print(f"Precision - Train: {train_precision:.4f}, Validation: {val_precision:.4f}")
    print(f"Recall - Train: {train_recall:.4f}, Validation: {val_recall:.4f}")
    print(f"F1 Score - Train: {train_f1:.4f}, Validation: {val_f1:.4f}")
    print("\nRapport de classification sur l'ensemble de validation:")
    print(classification_report(y_val, y_val_pred))
    
    # Matrice de confusion
    cm = confusion_matrix(y_val, y_val_pred)
    
    # Visualisation de la matrice de confusion avec Plotly
    fig = px.imshow(cm, 
                    labels=dict(x="Prédiction", y="Réalité", color="Nombre"),
                    x=['Non satisfait', 'Satisfait'],
                    y=['Non satisfait', 'Satisfait'],
                    text_auto=True,
                    title=f"Matrice de confusion - {model_name}",
                    color_continuous_scale='Blues')
    fig.update_layout(
        title_text=f"Matrice de confusion - {model_name}",
        xaxis_title="Prédiction",
        yaxis_title="Réalité",
        coloraxis_colorbar=dict(title="Nombre"),
        height=800,
        width=800
    )
    fig.show()
    
    # Courbe ROC si le modèle peut prédire des probabilités
    if hasattr(model, "predict_proba"):
        y_val_proba = model.predict_proba(X_val)[:, 1]
        fpr, tpr, _ = roc_curve(y_val, y_val_proba)
        roc_auc = auc(fpr, tpr)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'{model_name} (AUC = {roc_auc:.4f})'))
        fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Aléatoire', line=dict(dash='dash', color='gray')))
        
        fig.update_layout(
            title=f'Courbe ROC - {model_name}',
            xaxis_title='Taux de faux positifs',
            yaxis_title='Taux de vrais positifs',
            legend=dict(x=0.7, y=0.1),
            width=1000,
            height=800
        )
        fig.show()
    
    return {
        'model': model,
        'name': model_name,
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'train_precision': train_precision,
        'val_precision': val_precision,
        'train_recall': train_recall,
        'val_recall': val_recall,
        'train_f1': train_f1,
        'val_f1': val_f1
    }

## 5. Implémentation de XGBoost

XGBoost (eXtreme Gradient Boosting) est un algorithme puissant basé sur le gradient boosting qui est souvent utilisé dans les compétitions de machine learning en raison de ses performances élevées.

In [ ]:
# Création d'un modèle XGBoost de base
xgb_clf = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Évaluation du modèle XGBoost de base avec les données prétraitées standard
xgb_results = evaluate_model(xgb_clf, X_train_preprocessed, X_val_preprocessed, y_train, y_val, "XGBoost (Base)")

In [ ]:
# Évaluation du modèle XGBoost de base avec les données prétraitées et feature engineering
xgb_fe_results = evaluate_model(xgb_clf, X_train_fe_preprocessed, X_val_fe_preprocessed, y_train, y_val, "XGBoost (avec Feature Engineering)")

## 6. Optimisation des hyperparamètres de XGBoost

Nous allons maintenant optimiser les hyperparamètres de XGBoost pour améliorer ses performances. Nous utiliserons RandomizedSearchCV pour explorer efficacement l'espace des hyperparamètres.

In [ ]:
# Définition de l'espace des hyperparamètres à explorer
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 8],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0.1, 0.5, 1, 5]
}

# Création du modèle XGBoost pour l'optimisation
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Configuration de la recherche aléatoire
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=20,  # Nombre d'itérations (combinaisons d'hyperparamètres à tester)
    scoring='f1',  # Métrique à optimiser
    cv=3,  # Validation croisée à 3 plis
    verbose=1,
    random_state=42,
    n_jobs=-1  # Utiliser tous les cœurs disponibles
)

# Exécution de la recherche aléatoire sur les données avec feature engineering
print("Début de l'optimisation des hyperparamètres...")
random_search.fit(X_train_fe_preprocessed, y_train)
print("Optimisation terminée!")

In [ ]:
# Affichage des meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres trouvés:")
for param, value in random_search.best_params_.items():
    print(f"{param}: {value}")

print(f"\nMeilleur score F1: {random_search.best_score_:.4f}")

In [ ]:
# Visualisation des résultats de la recherche d'hyperparamètres
results = pd.DataFrame(random_search.cv_results_)

# Création d'un graphique pour visualiser l'impact des hyperparamètres sur le score F1
important_params = ['param_n_estimators', 'param_learning_rate', 'param_max_depth', 'param_subsample']
fig = make_subplots(rows=2, cols=2, subplot_titles=[param.replace('param_', '') for param in important_params])

for i, param in enumerate(important_params):
    row = i // 2 + 1
    col = i % 2 + 1
    
    # Conversion des paramètres en valeurs numériques pour le tracé
    param_values = results[param].astype(str).astype(float)
    
    fig.add_trace(
        go.Scatter(
            x=param_values,
            y=results['mean_test_score'],
            mode='markers',
            marker=dict(size=10, color=results['mean_test_score'], colorscale='Viridis', showscale=False),
            text=results['rank_test_score'],
            name=param.replace('param_', '')
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text=param.replace('param_', ''), row=row, col=col)
    fig.update_yaxes(title_text='Score F1', row=row, col=col)

fig.update_layout(
    title_text="Impact des hyperparamètres sur le score F1",
    height=800,
    width=1000,
    showlegend=False
)

fig.show()

In [ ]:
# Création d'un modèle XGBoost avec les meilleurs hyperparamètres
best_xgb = xgb.XGBClassifier(**random_search.best_params_, random_state=42, use_label_encoder=False, eval_metric='logloss')

# Évaluation du modèle XGBoost optimisé
best_xgb_results = evaluate_model(best_xgb, X_train_fe_preprocessed, X_val_fe_preprocessed, y_train, y_val, "XGBoost (Optimisé)")

## 7. Analyse des caractéristiques importantes

XGBoost nous permet d'analyser l'importance des différentes caractéristiques dans le modèle.

In [ ]:
# Obtention des noms des caractéristiques après transformation
# Pour les caractéristiques numériques
numeric_features_transformed = numeric_features_fe

# Pour les caractéristiques catégorielles (après one-hot encoding)
categorical_features_transformed = []
for cat_feature in categorical_features_fe:
    # Obtention des catégories uniques pour chaque caractéristique catégorielle
    unique_categories = X_train_fe[cat_feature].unique()
    # Création des noms de colonnes après one-hot encoding (en excluant la première catégorie)
    for category in unique_categories[1:]:
        categorical_features_transformed.append(f"{cat_feature}_{category}")

# Combinaison des noms de caractéristiques
all_features_transformed = numeric_features_transformed + categorical_features_transformed

# Vérification de la longueur
print(f"Nombre de caractéristiques après transformation: {len(all_features_transformed)}")
print(f"Forme des données prétraitées: {X_train_fe_preprocessed.shape[1]}")

# Ajustement si nécessaire (si les dimensions ne correspondent pas)
if len(all_features_transformed) != X_train_fe_preprocessed.shape[1]:
    print("Attention: Le nombre de noms de caractéristiques ne correspond pas à la dimension des données prétraitées.")
    # Utilisation d'indices numériques si les noms ne correspondent pas
    all_features_transformed = [f"Feature_{i}" for i in range(X_train_fe_preprocessed.shape[1])]

In [ ]:
# Extraction de l'importance des caractéristiques du modèle XGBoost optimisé
feature_importance = best_xgb.feature_importances_

# Création d'un DataFrame pour visualiser l'importance des caractéristiques
importance_df = pd.DataFrame({
    'Feature': all_features_transformed,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

# Affichage des 20 caractéristiques les plus importantes
top_20_features = importance_df.head(20)
top_20_features

In [ ]:
# Visualisation des 20 caractéristiques les plus importantes
fig = px.bar(top_20_features, 
             x='Importance', 
             y='Feature', 
             orientation='h',
             title='Top 20 des caractéristiques les plus importantes (XGBoost optimisé)',
             color='Importance',
             color_continuous_scale='Viridis')

fig.update_layout(
    xaxis_title='Importance',
    yaxis_title='Caractéristique',
    height=800,
    width=1000
)

fig.show()

## 8. Comparaison avec d'autres modèles avancés

Comparons maintenant XGBoost avec d'autres modèles avancés comme LightGBM et CatBoost.

In [ ]:
# Importation des bibliothèques pour LightGBM
import lightgbm as lgb

# Création d'un modèle LightGBM
lgb_clf = lgb.LGBMClassifier(random_state=42)

# Évaluation du modèle LightGBM
lgb_results = evaluate_model(lgb_clf, X_train_fe_preprocessed, X_val_fe_preprocessed, y_train, y_val, "LightGBM")

In [ ]:
# Création d'un modèle AdaBoost
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(random_state=42)

# Évaluation du modèle AdaBoost
ada_results = evaluate_model(ada_clf, X_train_fe_preprocessed, X_val_fe_preprocessed, y_train, y_val, "AdaBoost")

## 9. Comparaison finale des modèles

Comparons maintenant tous les modèles que nous avons implémentés.

In [ ]:
# Création d'un DataFrame pour comparer les modèles
results = [xgb_results, xgb_fe_results, best_xgb_results, lgb_results, ada_results]
comparison_df = pd.DataFrame({
    'Modèle': [result['name'] for result in results],
    'Accuracy (Train)': [result['train_accuracy'] for result in results],
    'Accuracy (Validation)': [result['val_accuracy'] for result in results],
    'Precision (Validation)': [result['val_precision'] for result in results],
    'Recall (Validation)': [result['val_recall'] for result in results],
    'F1 Score (Validation)': [result['val_f1'] for result in results]
})

comparison_df

In [ ]:
# Visualisation des performances des modèles
metrics = ['Accuracy (Validation)', 'Precision (Validation)', 'Recall (Validation)', 'F1 Score (Validation)']
model_names = comparison_df['Modèle']

fig = go.Figure()

for metric in metrics:
    fig.add_trace(go.Bar(
        x=model_names,
        y=comparison_df[metric],
        name=metric
    ))

fig.update_layout(
    title='Comparaison des performances des modèles avancés',
    xaxis_title='Modèle',
    yaxis_title='Score',
    barmode='group',
    width=1000,
    height=600
)

fig.show()

In [ ]:
# Visualisation de l'écart entre les performances d'entraînement et de validation (pour détecter le surapprentissage)
comparison_df['Écart Accuracy'] = comparison_df['Accuracy (Train)'] - comparison_df['Accuracy (Validation)']

fig = px.bar(comparison_df, 
             x='Modèle', 
             y='Écart Accuracy',
             title='Écart entre Accuracy d\'entraînement et de validation (indicateur de surapprentissage)',
             color='Écart Accuracy',
             color_continuous_scale='RdYlGn_r')

fig.update_layout(
    xaxis_title='Modèle',
    yaxis_title='Écart d\'Accuracy (Train - Validation)',
    height=500,
    width=900
)

fig.show()

## 10. Conclusion

Dans ce notebook, nous avons exploré des modèles de machine learning plus avancés, en nous concentrant particulièrement sur XGBoost avec optimisation des hyperparamètres. Nous avons également comparé XGBoost avec d'autres algorithmes de boosting comme LightGBM et CatBoost.

### Points clés à retenir :

1. **Feature Engineering** : L'ajout de nouvelles caractéristiques a permis d'améliorer les performances des modèles.

2. **Optimisation des hyperparamètres** : L'optimisation des hyperparamètres de XGBoost a conduit à une amélioration significative des performances.

3. **Importance des caractéristiques** : L'analyse de l'importance des caractéristiques nous a permis d'identifier les facteurs les plus déterminants pour la satisfaction des passagers.

4. **Comparaison des modèles** : Les modèles de boosting (XGBoost, LightGBM, CatBoost) ont généralement montré de meilleures performances que les modèles plus simples.

5. **Surapprentissage** : Certains modèles ont montré des signes de surapprentissage, avec un écart important entre les performances d'entraînement et de validation.

### Prochaines étapes :

- Utiliser la validation croisée pour obtenir une estimation plus robuste des performances des modèles
- Explorer des techniques d'ensemble learning pour combiner les prédictions de différents modèles
- Tester les modèles sur l'ensemble de test pour évaluer leur généralisation
- Déployer le meilleur modèle dans un environnement de production